# 自动发送邮件

使用Python内置的smtplib和email两个模块，实现邮件的自动发送。

## 配置邮箱
在使用Python发送邮件之前我们需要获取发件邮箱的认证，在获取认证这一块我们需要两步：

第一步：配置发件邮箱账户以及密码

第二步：根据根据邮箱获取授权码

这两步都需要在电子邮箱的账户配置中设置，这里我们以QQ邮箱为例进行配置。

首先，登陆邮箱找到设置-->账户：

然后，向下滑动找到POP3开头的选项，并开启下图中两个选项，然后会有授权码，请将其保存好：
配置好以后，我们就可以使用Python进行设置邮件的内容，最后发送邮件了。

##  了解smtplib和email
一、SMTP是Python中用于由源地址到目的地址传送邮件的传输协议。smtplib模块对该协议进行封装，从而提供了一种很方便的发送电子邮件的方法。

    - 第一步，在发送邮件之前，创建一个服务对象。

import smtplib
server = smtplib.SMTP_SSL(host , port )
    
    host：SMTP服务器主机。

    port：指定 SMTP 服务使用的端口号，一般情况下 SMTP 端口号为25, ssl加密传输的话端口就是465

    - 第二步，登陆服务器邮箱：

server.login(from_addr, password)
    
    from_addr：发件人邮箱地址。

    password：邮箱密码，配置时的授权码。

    - 第三步，发送邮件：

server.sendmail(from_addr, [to_addr], message.as_string())

    from_addr：发件人的邮箱地址。

    to_addr：收件人的邮箱（列表形式）。

    message.as_string()：把邮件内容MIMEText对象变成str。
    
二、email模块主要负责构造邮件，例如，邮件头信息中的发件人、收件人、以及主题和时间等，邮件体信息中的附件、使用html书写邮件、图片等等，我们都需要使用email模块。

In [ ]:
# 用来发送邮件，包括构建邮件服务、登录、发送、退出等动作
import smtplib
# 构建邮件头信息，包括发件人、接收人、标题等
from email.mime.multipart import MIMEMultipart
# 构建邮件正文，可以使text或HTML
from email.mime.text import MIMEText
# 构建邮件标题的，可以支持标题中文
from email.header import Header
# 构建邮件附件
from email.mime.application import MIMEApplication

### 第一步，邮箱配置

In [ ]:
# 发送邮箱地址
from_addr = '1562268871@qq.com'

# 邮箱授权码，非登录密码
password = ''

# 收件箱地址，可以发多人
to_addr = ['','','']

# 邮箱服务器地址SMTPserver
smtp_server = 'smtp.qq.com'



### 第二步，邮件内容设置

In [ ]:
# 创建一个邮件信息的对象
message = MIMEMultipart()

# 发送邮箱地址
message['From'] = from_addr

#收件箱地址
message['To'] = ';'.join(to_addr)

#主题
message['Subject'] = '邮件标题'

# 分别为邮件内容MIME的subtype类型编码格式
content_text = MIMEText('测试邮件文本','plain','utf-8')

message.attach(content_text)

### 第三步，邮件发送

In [ ]:
try:
    # SMTP协议加密端口是465
    server = smtplib.SMTP_SSL(smtp_server,465)
    
    # 登录服务器邮箱
    server.login(from_addr,password)
    
    # 发送邮件
    server.sendmail(from_addr,to_addr,message.as_string())
    
    # 退出服务器邮箱
    server.quit()
    
    print('邮件发送成功')
    
except smtplib.SMTPException as e:
    server.quit()
    print('error',e)
    

##  发送带有附件的邮件

In [ ]:
from email.mime.image import MIMEImage
import os

# 创建一个邮件信息的对象
message = MIMEMultipart()

# 发送邮箱地址
message['From'] = from_addr

#收件箱地址
message['To'] = ';'.join(to_addr)

#主题
message['Subject'] = '邮件标题'

# 附件本地路径
imageFile = 'E:/Jupyter-notebook/数据分析基础/output/a.png'

# 添加图片附件内容
with open(imageFile,'rb') as fb:
    imageApart = MIMEImage(fp.read())

# 设置附件头信息，filename的设置是解决中文乱码问题
imageApart.add_header('Content-Disposition','attachment',filename=('utf-8','',os.path.basename(imageFile)))

# 将内容附加到邮件主体中
message.attach(imageApart)

## 发送html格式的邮件

In [ ]:
# 创建一个邮件信息的对象
message = MIMEMultipart()

# 发送邮箱地址
message['From'] = from_addr

# 收件箱地址
message['To'] = ';'.join(to_addr)

# 主题
message['Subject'] = '邮件标题'

# 推荐使用html格式的正文内容，这样比较灵活，可以附加图片地址，调整格式等
with open('./output/bar1.html','rb') as f:
    content = f.read()
    #设置html格式参数
    html_part = MIMEText(content,'html','utf-8')

message.attach(html_part)

## html格式的邮件需要注意的问题

    1. 由于安全问题，邮箱判断html代码中的<script>不执行，所以需要解决自动添加时间的问题 
      解决思路：利用 python 读取html时进行文本替换 

    2. 发送html格式中带有本地图片的邮件，不能再html代码中直接加载显现
      解决办法：需要将图片以附件的形式添加到邮件信息中。html直接读取附件 


In [ ]:
# 添加时间（文本替换）
with open('./output/bar1.html','r',encoding='utf-8') as file:
    content = file.readlines()

# 替换的时候需要注意，文本在读取出来的时候，每一行的开始或者是结尾 ，会存在 \n \t 这些字符,我们可以利用分行读取，分行处理的方式

content_list = []
for line in content:
    content_list.append(line.strip())

content = ''.join(content_list)

# 将时间替换（由于html 被作为邮件的形式发送的时候js代码不执行）

import datetime
dateTime_p = datetime.datetime.now()
str_p = datetime.datetime.strftime(dateTime_p,'%Y年%m月%d日')

content = content.replace('now_self',str_p)

with open('./日报.html','w',encoding='utf-8') as file:
    file.write(content))

## 完整发送邮件

In [ ]:
# 用来发送邮件，包括构建邮件服务、登录、发送、退出等动作
import smtplib
# 构建邮件头信息，包括发件人、接收人、标题等
from email.mime.multipart import MIMEMultipart
# 构建邮件正文，可以使text或HTML
from email.mime.text import MIMEText
# 构建邮件标题的，可以支持标题中文
from email.header import Header
# 构建邮件附件
from email.mime.application import MIMEApplication


# 发送邮箱地址
from_addr = '1562268871@qq.com'

# 邮箱授权码，非登录密码
password = ''

# 收件箱地址，可以发多人
to_addr = ['','','']

# 邮箱服务器地址SMTPserver
smtp_server = 'smtp.qq.com'

# 创建一个邮件信息的对象
message = MIMEMultipart()

# 发送邮箱地址
# message['From'] = from_addr

# 解决收件箱中发件人名字
from email.utils import formataddr
message['From'] = formataddr(('panda',from_addr))

# 收件箱地址
message['To'] = ';'.join(to_addr)

# 主题
message['Subject'] = '邮件标题'

with open('./output/bar1.html','rb') as f:
    #设置html格式参数
    html_part = MIMEText(f.read(),'html','utf-8')
message.attach(html_part)

# 设置正文图片(需要设置 图片与 html 的对应上)
part_picture = MIMEApplication(open('./image/1.png','rb').read())
part_picture.add_header('Content-ID','image01') #设置图片ID，插入正文
message.attach(part_picture)


# 邮件发送
try:
    # SMTP协议加密端口是465
    server = smtplib.SMTP_SSL(smtp_server,465)
    
    # 登录服务器邮箱
    server.login(from_addr,password)
    
    # 发送邮件
    server.sendmail(from_addr,to_addr,message.as_string())
    
    # 退出服务器邮箱
    server.quit()
    
    print('邮件发送成功')
    
except smtplib.SMTPException as e:
    server.quit()
    print('error',e)
